    Hypotheses set for this Linear Regression model 
    Hypothesis 1: Houses has more rooms have higher price ('Room')
    Hypothesis 2: Houses type 'h' has higher price than houses type 't', and houses type 't' have higher price than houses type 'u' ('Type')
    Hypothesis 3: Houses in the central with smaller distance has higher price ('Distance')
    Hypothesis 4: Houses has more parking space has higher price ('Car')
    Hypothesis 5: Houses has larger land size has higher price ('Landsize')
    Hypothesis 6: Houses has larger building area has higher price ('Building Area')
    Hypothesis 7: Houses has long year used has lower price ('YearUsed')
    Hypothesis 8: Houses is used more than 60 years and above has higher price ('Heritage')
    Hypothesis 9: Houses price is different depend on the region

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
pd.set_option('float_format', '{:,.2f}'.format)
pd.set_option('max_columns', None)

I. Data cleaning

In [ ]:
data = pd.read_csv('Downloads/Melbourne_housing_FULL.csv')

In [ ]:
data

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

Variable 'Price' has 7610 missing values, accounting for ~22% total value. If consider these missing values by variable 'Method' value, we can conclude that there are 2 cases for the reason:
1. Houses autionned can't match the price in the auction (PI, VB) or bidder withdrawn prior to auction (W) 
2. Houses sold not via auction (SP, SN, PN, SS, SA) or via auction but no data found (S)
The first case has 2192 observation, accounting for ~6,29% total value. The second case has 5418 observation, accounting for ~ 15,54% observation. For the first case, we will drop all the missing values because these are incomplete transaction. For missing values in the second case, we will replace them by mean price value of 'Regionname' of that house.
Ex: A house with missing 'Price' value, 'Method' type 'SP', 'Regionname' 'Sounthern Metropolian' will have a price value of 1,395,928.33

In [ ]:
#Mean price value by region name
mean_by_region = data.groupby('Regionname').Price.mean().sort_values(ascending = False)
mean_by_region

In [ ]:
#Drop missing value price cells with method is either ['PI', 'VB', 'W']
data_2 = data[~((data.Price.isnull() == True) & (data.Method.isin(['PI', 'VB', 'W'])))]

In [ ]:
#replace missing value price cells with mean_by_region value for method is either in the second case
for region in mean_by_region.index:
    data_2.loc[(data_2.Price.isnull() == True) & (data_2.Regionname == region), 'Price'] = mean_by_region[region]

In [ ]:
#drop missing value in column with small contribution
data_2 = data_2.dropna(subset = ['Distance', 'Postcode', 'CouncilArea', 'Regionname', 'Propertycount'])

In [ ]:
#drop column which play no role in expecting a house's price
data_2 = data_2.drop(columns = ['Lattitude', 'Longtitude', 'Postcode', 'Propertycount'])

In [ ]:
data_2

See that there're still a lot missing values in other column, there are 2 solutions we can think of:
1. Use dropna right away
2. Keep replace missing values by mean values:
    + See that the correlation between 'Romms' and 'Bedroom2' is really hight (0,96) -> drop one, keep on. So we will drop 'Bedroom2'
    + Bathroom: same as above, drop 'Bathroom' also
    + Car: replace missing value by 'Type'. House type 'h' will have more parking space than house type 't' or 'u'
    + Landsize: same as above
    + Buildingarea: same as above
    + YearBuild: Base on mean year build by 'Regionname' and 'Type'

In [ ]:
data_3 = data_2.dropna()

In [ ]:
#Conver 'Date' from str to Datetime
data_2.Date = pd.to_datetime(data_2.Date, format = '%d/%m/%Y')
data_3.Date = pd.to_datetime(data_3.Date, format = '%d/%m/%Y')

In [ ]:
#Correlation matrix between variables
data_2.corr()

- 'Price' has pretty high correlated value with 'Room', 'Distance', 'Bedroom2', 'Bathroom', 'YearBuilt'
- 'Rooms' has really high correlated value with 'Bedroom2', 'Car' and 'Bedroom', so multicollinearity may happens between these 3 variables

In [ ]:
mean_car = data_2.groupby('Type').Car.mean()
mean_car

In [ ]:
mean_BA = data_2.groupby('Type').BuildingArea.mean()
mean_BA

In [ ]:
mean_LS = data_2.groupby('Type').Landsize.mean()
mean_LS

In [ ]:
mean_YB = data_2.groupby(['Regionname','Type']).YearBuilt.mean()
mean_YB

In [ ]:
#drop 'Bedroom2' and 'Bathroom' to avoid multicollinearity
data_2 = data_2.drop(columns = ['Bedroom2', 'Bathroom'])

In [ ]:
#replace missing value 'Car'
for house_type in mean_car.index:
    data_2.loc[(data_2.Car.isnull() == True) & (data_2.Type == house_type), 'Car'] = mean_car[house_type]

In [ ]:
#replace missing value 'BuildingArea'
for house_type in mean_BA.index:
    data_2.loc[(data_2.BuildingArea.isnull() == True) & (data_2.Type == house_type), 'BuildingArea'] = mean_BA[house_type]

In [ ]:
#replace missing value 'Landsize'
for house_type in mean_LS.index:
    data_2.loc[(data_2.Landsize.isnull() == True) & (data_2.Type == house_type), 'Landsize'] = mean_LS[house_type]

In [ ]:
#replace missing value 'YearBuilt'
for region in mean_YB.index.levels[0]:
    for house_type in mean_YB[region].index:
        data_2.loc[(data_2.YearBuilt.isnull() == True) & (data_2.Type == house_type) & (data_2.Regionname == region), 'YearBuilt'] = mean_YB[region][house_type]

In [ ]:
data_2.isnull().sum()

In [ ]:
#Still there are 3 observations of 'YearBuilt' with Null values, we decide to drop it since it just take a small amount
data_2 = data_2.dropna()

In [ ]:
data_2.index = range(data_2.shape[0])

II. Data exploration

In [ ]:
data_2.describe()

- Mean price is at 1,067,376.83 with stanard deviation at 596,099.44
- Mean price is pretty close to average price at 50%: 900,00.00 => standard distribution
- Max price is at 11,200,000.00
- Most of the price oscillate from 85,000.00 (min) to 1,395,928.33 (75%)

In [ ]:
data_2.groupby('Rooms').Price.describe().sort_values('mean', ascending = False)

- Houses have more room has higher price
- But there's an exception: houses have about 7 - 9 rooms has its price dropped
- This is partly because these houses don't have enough representative pattern
- Houses with 3 rooms accounting for the most transaction, up to 14,271
- Houses with 7 rooms and upper has a really low transaction rate, end up at 30 or even lower
=> Hypothesis 1: Houses has more rooms have higher price ('Room')

In [ ]:
data_2.groupby('Type').Price.describe().sort_values('mean', ascending = False)

- Houses type 'h' (cottage, villa, house, semi, terrace) have mean price at $1,192,671.09, higher than houses type 't'(townhouse) and 'u' (unit, duplex) 
=> Hypothesis 2: Houses type 'h' has higher price than houses type 't', and houses type 't' have higher price than houses type 'u' ('Type')

In [ ]:
data_2.groupby('Method').Price.describe().sort_values('mean', ascending = False)

- Auction method doesn't really have a significant impact on house's price, since these means really close to one another

In [ ]:
data_2.groupby(data_2.Date.dt.year).Price.describe().sort_values('mean', ascending = False)

- Houses price from 2016 to 2018 don't really have a significant change

In [ ]:
#Since 'Distance' is quite a distributed values, we could group it by a arbitrary range, here is every 5 unit
dis = [-0.1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
dis = pd.cut(data_2.Distance, dis)
data_2.groupby(dis).Price.agg(['count', 'mean'])

- Houses with distance from 15 and lower own the highest price
- The farther (from the central), the lower the price
- Most of the transaction come from houses with distance is from 20 and lower
=> Hypothesis 3: Houses in the central with smaller distance has higher price ('Distance')

In [ ]:
#Since 'Car' is quite a distributed values, we could group it by a arbitrary range, here is every 5 unit
car = [-0.1, 1, 2, 4, 6, 8, 10, 30]
car = pd.cut(data_2.Car, car)
data_2.groupby(car).Price.agg(['count', 'mean'])

- In general, houses with more parking space (car) tend to have higher price, but this assumption is converse when the parking space value become to high
=> Hypothesis 4: Houses has more parking space has higher price ('Car')

In [ ]:
#Since 'Landsize' is quite a distributed values, we could group it by a arbitrary range, here is every 5 unit
ls = [-0.1, 150, 250, 350, 450, 550, 650, 750, 450000]
ls = pd.cut(data_2.Landsize, ls)
data_2.groupby(ls).Price.agg(['count', 'mean'])

In general observation, houses with larger land size has a higher price
=> Hypothesis 5: Houses has larger land size has higher price ('Landsize')

In [ ]:
#Since 'BuildingArea' is quite a distributed values, we could group it by a arbitrary range, here is every 5 unit
ba = [-0.1, 50, 150, 200, 250, 3200]
ba = pd.cut(data_2.BuildingArea, ba)
data_2.groupby(ba).Price.agg(['count', 'mean'])

In general observation, houses with larger building area has a higher price
=> Hypothesis 6: Houses has larger building area has higher price ('Building Area')

To evaluate the quality of the 'YearBuilt' variable, we create a variable 'YearUsed' by subtract date sale 'Date' for date built 'YearBuilt' 

In [ ]:
data_2['YearUsed'] = data_2.Date.dt.year - data_2['YearBuilt']
data_3['YearUsed'] = data_3.Date.dt.year - data_3['YearBuilt']

See that there are some 'YearUsed' with negative value ( <0 ), we assume that this is a typo in data entry or these house are sold before having built. We will replace these value by 0

In [ ]:
data_2.loc[data_2.YearUsed<0, 'YearUsed'] = 0
data_3.loc[data_3.YearUsed<0, 'YearUsed'] = 0

In [ ]:
yu = [-0.1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 850]
yu = pd.cut(data_2.BuildingArea, yu)
data_2.groupby(yu).Price.agg(['count', 'mean'])

In general we can see that a house with a longer time used has its price drop down.
But house with over 60 years used has a higher price due to it heritage aspect. We will create a dummy variable 'Heritage' for this case. If YearUsed > 60, Heritage is equal to 1 and equal to 0 when YearUsed < 60
-> Hypothesis 7: Houses has long year used has lower price ('YearUsed')
-> Hypothesis 8: Houses is used more than 60 years and above has higher price ('Heritage')

In [ ]:
#Create Heritage variable
data_2['Heritage'] = np.where(data_2.YearUsed > 60, 1, 0)
data_3['Heritage'] = np.where(data_3.YearUsed > 60, 1, 0)

In [ ]:
data_2.groupby('Regionname').Price.describe().sort_values('mean', ascending = False)

House price has a different depends on region, reach its highest at $1.395.928 at Southern Metropolitan
-> Hypothesis 9: Houses price is different depend on the region

In [ ]:
#data_2 - Type
one_hot = pd.get_dummies(data_2.Type)
data_2 = data_2.join(one_hot)
data_2 = data_2.drop(columns = ['Type', 'u'])

#data_2 - Regionname
one_hot = pd.get_dummies(data_2.Regionname)
data_2 = data_2.join(one_hot)
data_2 = data_2.drop(columns = ['Regionname', 'Western Victoria'])

#data_3 - Type
one_hot = pd.get_dummies(data_2.Type)
data_3 = data_3.join(one_hot)
data_3 = data_3.drop(columns = ['Type', 'u'])

#data_3 - Regionname
one_hot = pd.get_dummies(data_2.Type)
data_3 = data_3.join(one_hot)
data_3 = data_3.drop(columns = ['Regionname', 'Western Victoria'])

In [ ]:
data.columns = data.columns.str.strip()

In [ ]:
#drop columns that has no use for analystical
data2 = data_2.drop(columns = ['Suburb', 'Address', 'Method', 'SellerG', 'Date', 'YearBuilt', 'CouncilArea'])
data3 = data_3.drop(columns = ['Suburb', 'Address', 'Method', 'SellerG', 'Date', 'YearBuilt', 'CouncilArea'])

III. Holdout & data valiadation

In [ ]:
#Create x and y set use in regression
x_model2 = data_2.drop(columns = ['Price'])
y_model2 = (data_2.Price)/1000

x_model3 = data_3.drop(columns = ['Price'])
y_model3 = (data_3.Price)/1000

In [ ]:
#regression OLS
lin_reg = LinearRegression()
x_model2_train, x_model2_test, y_model2_train, y_model2_test = train_test_split(x_model2, y_model2, test_size = 0.2, random_state = 0)
lin_reg.fit(x_model2_train, y_model2_train)

#make predict on test sample
y_model2_pred = lin_reg.predict(x_model2_test)
residuals_model2 = y_model2_pred - y_model2_test

#print out summary report using statsmodels module
x_model2_train2 = sm.add_constant(x_model2_train)
est = sm.OLS(y_model2_train, x_model2_train2)
est2 = est.fit()
print(est2.summary())

In [ ]:
lin_reg.score(x_model2_test, y_model2_test) 

lin_reg score is a negative value, means that our model has something wrong
it could be that the model doesn't fit well with test set, or there's outlier in test set cause this
-> draw a scatter plot to find out

In [ ]:
residual_ytest_model2 = pd.DataFrame({'residual': residuals_model2, 'y_pred': y_model2_pred})
residual_ytest_model2.plot(kind = 'scatter', x = 'y_pred', y = 'residual', figsize = (20,10))

See that there are an observation with a really big value of residual regression index = 21472, we'll try to remove it to see the fit ability of the model on test set

In [ ]:
residual_ytest_model2 = residual_ytest_model2[~(residual_ytest_model2.residual == residuals_model2.max())]
residual_ytest_model2.plot(kind = 'scatter', x = 'y_pred', y = 'residual', figsize = (20,10))
lin_reg.score(x_model2_test.drop(21472, axis = 0), y_model2_test.drop(21472, axis = 0)) 

After removed the outlier, the model fit well with R2 = 0,44. We will remove it from the dataset before hand on other analysis
From the scatter chart, we can see that the residual has a heterogeneous variance. House with low price has small residual relatively. House with higher price has big and distributed residual 

Similarly, we run regression OLS on data_3 (the one we dropna completely)

In [ ]:
#regression OLS
lin_reg2 = LinearRegression()
x_model3_train, x_model3_test, y_model3_train, y_model3_test = train_test_split(x_model3, y_model3, test_size = 0.2, random_state = 0)
lin_reg2.fit(x_model3_train, y_model3_train)

#make predict on test sample
y_model3_pred = lin_reg2.predict(x_model3_test)
residual_model3 = y_model3_pred - y_model3_test

#print out summary report using statsmodels module
x_model3_train2 = sm.add_constant(x_model3_train)
est = sm.OLS(y_model3_train, x_model3_train2)
est2 = est.fit()
print(est2.summary())

In [ ]:
lin_reg2.score(x_model3_test, y_model3_test) 

In [ ]:
#drop the outlier value with index = 21472
x_model2 = x_model2.drop(index = 21472)
x_model3 = x_model3.drop(index = 21472)

Apply model with cross validation

In [ ]:
#apply OLS model with cross validation on data_2
sum_R2 = 0
count_R2 = 0
for test in range(10):
    lin_Reg = LinearRegression()
    x_model2_train, x_model2_test, y_model2_train, y_model2_test = train_test_split(x_model2, y_model2, test_size = 0.2)
    lin_reg.fit(x_model2_train, y_model2_train)
    sum_R2 += lin_reg.score(x_model2_test, y_model2_test)
    count R2 += 1
    prince('# R2 score of test %d: %.2f' %(test, lin_Reg.score(x_model2_test, y_model2_test)))
print('# Average R2 score: %.2f' %(sum_R2/ count_R2))

Average fit ability of OLS model toward data_2 reach 0,50

In [ ]:
#apply OLS model with cross validation on data_3
sum_R2 = 0
count_R2 = 0
for test in range(10):
    lin_reg = LinearRegression()
    x_model3_train, x_model3_test, y_model3_train, y_model3_test = train_test_split(x_model3, y_model3, test_size = 0.2)
    lin_reg.fit(x_model3_train, y_model3_train)
    sum_R2 += lin_reg.score(x_model3_test, y_model3_test)
    count_R2 += 1
    print(' # R2 score of %d test: %.2f' %(test, lin_reg.score(x_model3_test, y_model3_test)))
print('# Average R2 score: %.2f' %(sum_R2/count_R2))

Average fit ability of OLS model toward data_3 reach 0,55

Conclusion: Regression model OLS has a stastical meaning when p-value of its variables is very small and p-value of F-test approximately to 0. However we can not tell much from the model because R2 score is about 0,5 - 0,55, means that the model can just explain almost a half the fluctuation of the house price. Residual scatter plot also showed that regression residual has homoscedasticity (hiện tượng phương sai k đồng nhất). Correlation between independant variables signal of multicollinearity
This could be contributed by a removed variable 'SellerG' that hasn't been clearly explained and analyzied

IV. PCA Regression

In [ ]:
#Determine the dimension need to be reduced in PCA on data_2
pca = PCA()
x_model2_reduced = pca.fit_transform(scale(x_model2))
pca_var_explain = pd.Series(np.round(pca.explained_variance_ratio_ *100, decimals=2))
pca_var_explain.plot(kind = 'bar')
pca_var_explain.cumsum()

From the result, we see that when k = 10, we could obtain 92% information

In [ ]:
#Determine the dimension need to be reduced in PCA on data_3
pca_data3 = PCA()
x_model3_reduced = pca_data3.fit_transform(scale(x_model3))
pca_var_explain_data3 = pd.Series(np.round(pca.explained_variance_ratio_ *100, decimals=2))
pca_var_explain_data3.plot(kind = 'bar')
pca_var_explain_data3.cumsum()

From the result, we see that when k = 10, we could obtain 90,64% information

In [ ]:
#PCA Regression on data_2
sum_R2 = 0
count_R2 = 0
x_model2_reduced = PCA(n_components = 10).fit_transform(scale(x_model2))
for test in range(10):
    lin_reg = LinearRegression()
    x_model2_train, x_model2_test, y_model2_train, y_model2_test = train_test_split(x_model2_reduced, y_model2, test_size = 0.2)
    lin_reg.fit(x_model2_train, y_model2_train)
    sum_R2 += lin_reg.score(x_model2_test, y_model2_test)
    count R2 += 1
    print('# R2 score of %d test: %.2f' %(test, lin_reg.score(x_model2_test, y_model2_test)))
print('# Average R2 score: %.2f' %(sum_R2/count_R2))

The result doesn't bring much change compare to Linear Regression model

In [ ]:
#PCA Regression on data_3
sum_R2 = 0
count_R2 = 0
x_model3_reduced = PCA(n_components = 10).fit_transform(scale(x_model3))
for test in range(10):
    lin_reg = LinearRegression()
    x_model3_train, x_model3_test, y_model3_train, y_model3_test = train_test_split(x_model3_reduced, y_model3, test_size = 0.2)
    lin_reg.fit(x_model3_train, y_model3_train)
    sum_R2 += lin_reg.score(x_model3_test, y_model3_test)
    count R2 += 1
    print('# R2 score of %d test: %.2f' %(test, lin_reg.score(x_model2_test, y_model2_test)))
print('# Average R2 score: %.2f' %(sum_R2/count_R2))

Conclusion: The regression result after applied PCA doesn't have much improvement because its dimension values doesn't have an outstanding value compare to the others

In [ ]:
V. Tuning model

We tuning our model because we have 2 problems: a variable 'SellerG' is left out, and homoscedasticity 
We add in 'SellerG' using a ordinal variable, since the original has type 'category', called 'SellerG_idx' with value [1,2,3,4] when the average sell price of the seller fall into ranges: [min_,Q1), [Q1,Q2), [Q2,Q3), [Q3,max_)
In addition, we take logarit of 'Price', 'Distance', 'Landsize' and 'BuildingArea' in logarit regression

In [ ]:
#Remove variable that has no use in our model and drop the outlier value
data_2 = data_2.drop(columns = ['Suburb', 'Method', 'Date', 'YearBuilt', 'CouncilArea'])
data_2 = data_2.drop(index = 21472)

In [ ]:
#reindex data
data_2 = range(data_2.shape[0])

In [ ]:
data_2.groupby('SellerG').Price.mean().describe

In [ ]:
#milestone for encoding SellerG
min_ = data_2.groupby('SellerG').Price.mean().min()
Q1_ = data_2.groupby('SellerG').Price.mean().quantile(0.25)
Q2_ = data_2.groupby('SellerG').Price.mean().quantile(0.5)
Q3_ = data_2.groupby('SellerG').Price.mean().quantile(0.75)
max_ = data_2.groupby('SellerG').Price.mean().max()

In [ ]:
#encode SellerG
temp = pd.Series(data_2.groupby('SellerG').Price.mean())
mean_price_by_seller = pd.DataFrame({'SellerG': temp.index, 'mean_price':temp.values, 'SellerG_idx':0})
for i in range(len(temp.index)):
    mean_price = mean_price_by_seller.loc[i, 'mean_price']
    sellerg_idx = mean_price_by_seller.loc[i, 'SellerG_idx']
    if mean_price < Q1
        mean_price_by_seller.loc[i, 'SellerG_idx'] = 1
    elif mean_price < Q2:
        mean_price_by_seller.loc[i, 'SellerG_idx'] = 2
    elif mean_price < Q3:
        mean_price_by_seller.loc[i, 'SellerG_idx'] = 3
    elif mean_price < Q4:
        mean_price_by_seller.loc[i, 'SellerG_idx'] = 4
        
for record in mean_price_seller.values:
    data_2.loc[data_2.SellerG == record[0], 'SellerG'] = record[2]

In [ ]:
# Logarit 'Price', 'Distance', 'Landsize', 'BuildingArea'
#Remove jamming with 0 values
data_2 = data_2[~(data_2.Distance == 0)]
data_2 = data_2[~(data_2.Langsize == 0)]
data_2 = data_2[~(data_2.BuildingArea == 0)]

#Take logarit
data_2.Price = np.log(data_2.Price)
data_2.Distance = np.log(data_2.Distance)
data_2.Landsize = np.log(data_2.Landsize)
data_2.BuildingArea = np.log(data_2.BuildingArea)

In [ ]:
x_model4 = data_2.drop(column=['Price', 'Address'])
y_model4 = (data_2.Price)/1000

In [ ]:
#OLS Regression on data_2
lin_reg = LinearRegression()
x_model4_train, x_model4_test, y_model4_train, y_model4_test = train_test_split(x_model4, y_model4, test_size = 0.2, randomstate = 0)
lin_reg.fit(x_model4_train, y_model4_train)

#make prediction on test sample
y_model4_pred = lin-reg.predict(x_model4_test)
residuals_model4 = y_model4_pred - y_model4_test

#print summary using statsmodels module
x_model4_train2 = sm.add_constant(x_model4_train)
est = sm.OLS(y_model4_train, x_model4_train2)
est2 = est.fit()
print(est2.summary())

In [ ]:
residual_ytest_model4 = pd.DataFrame({'residual:'residuals_model4, 'y_pred':y_model4_pred})
residual_ytest_model4.plot(kind = 'scatter', x ='y_pred', y ='residual', figsize=(20,10))

In [ ]:
#apply OLS model with cross validation on data_2
sum_R2 = 0
count_R2 = 0
for test in range(10):
    lin_reg = LinearRegression()
    x_model4_train, x_model4_test, y_model4_train, y_model4_test = train_test_split(x_model4_reduced, y_model4, test_size = 0.2)
    lin_reg.fit(x_model4_train, y_model4_train)
    sum_R2 += lin_reg.score(x_model4_test, y_model4_test)
    count R2 += 1
    print('# R2 score of %d test: %.2f' %(test, lin_reg.score(x_model4_test, y_model4_test)))
print('# Average R2 score: %.2f' %(sum_R2/count_R2))

Now we can tell that add 'SellerG' variable in and take logarit numeric variables improve the explain ability of our model. Apply cross validation on test sample result an average score at 0.64, much improvement since before tuning

In [ ]:
#Determine the dimension need to be reduced in PCA on data_2
pca = PCA()
x_model4_reduced = pca.fit_transform(scale(x_model4))
pca_var_explain = pd.Series(np.round(pca.explained_variance_ratio_ *100, decimals=2))
pca_var_explain.plot(kind = 'bar')
pca_var_explain.cumsum()

In [ ]:
#apply PCA regression on data_2
sum_R2 = 0
count_R2 = 0
x_model4_reduced = PCA(n_components = 10).fit_transform(scale(x_model4))
for test in range(10):
    lin_reg = LinearRegression()
    x_model4_train, x_model4_test, y_model4_train, y_model4_test = train_test_split(x_model4_reduced, y_model4, test_size = 0.2)
    lin_reg.fit(x_model3_train, y_model3_train)
    sum_R2 += lin_reg.score(x_model4_test, y_model4_test)
    count R2 += 1
    print('# R2 score of %d test: %.2f' %(test, lin_reg.score(x_model4_test, y_model4_test)))
print('# Average R2 score: %.2f' %(sum_R2/count_R2))